In [3]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn import linear_model
import lightgbm
import catboost
import xgboost as xgb
import shap

from catboost import *
%matplotlib
###########wrapping root mean square error for later calls##########
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=sum(absError)/len(absError)
    return mae,mse,RMSE
###########loading data##########
fdata=pd.read_csv('database_filled.csv',encoding="gbk")
raw_data=fdata.loc[:,[                      
                       'Zn Content (wt. %)',#0
                      'Co Cotent (wt. %)',#1
                      'Fe Cotent (wt. %)',#2
                      'O Content (at. %)',#4
                      'S Content (at. %)',#5
                      'Content of Pyridinic Species (at. %)',#5
                      'Content of Graphitic Species (at. %)',#6
                      'Content of Oxidized Species (at. %)',#7
                      'Raman ID/IG Ratio',#8
                      'BET Surface Area (m2 g-1)',#9
                      'Micropore Ratio',#10
                      'Mesopore Ratio',#11
                      'Catalyst Loading on RDE/RRDE(mg cm-2)',#12
                      'Rotating Speed (rpm)',#13
                      'Half Wave Potential in Acid Condition (vs. RHE)'#the classification target#14
                        ]]
###########train test splitting##########
raw_param=raw_data.iloc[:,0:14]
raw_power=raw_data.iloc[:,14]
###########fix random seed for reproducability##########
seed=71
X_train, X_test, y_train, y_test = train_test_split(raw_param, raw_power, test_size=.15,random_state=seed)
y_test_values=y_test.values.astype(np.float32)

########################################################
########################################################
###########################SHAP#########################
########################################################
########################################################

def shap_plot(model,param):
    print('start')
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    result = best_model.predict(X_test)
    x_prediction_07=result
    y_real_07=y_test_values
    x_prediction_07_series=pd.Series(x_prediction_07)
    y_real_07_series=pd.Series(y_real_07)
    ######SHAP#####
    data_SHAP = raw_data
    SHAP_INPUT=data_SHAP.iloc[:,0:14]
    SHAP_OUTPUT=data_SHAP.iloc[:,14]
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
    explainer = shap.TreeExplainer(best_model,X_SHAP)
    cols=[ 
                          'Zn Content (wt. %)',#0
                      'Co Cotent (wt. %)',#1
                      'Fe Cotent (wt. %)',#2
                      'O Content (at. %)',#4
                      'S Content (at. %)',#5
                      'Content of Pyridinic Species (at. %)',#5
                      'Content of Graphitic Species (at. %)',#6
                      'Content of Oxidized Species (at. %)',#7
                      'Raman ID/IG Ratio',#8
                      'BET Surface Area (m2 g-1)',#9
                      'Micropore Ratio',#10
                      'Mesopore Ratio',#11
                      'Catalyst Loading on RDE/RRDE(mg cm-2)',#12
                      'Rotating Speed (rpm)'
                            ]
    shap_values = explainer.shap_values(X_SHAP)
    shap.summary_plot(shap_values, SHAP_INPUT)
    print(type(shap_values))
    print('finished')
def shap_plot_CAT(model,param):
    print('start')
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    result = best_model.predict(X_test)
    x_prediction_07=result
    y_real_07=y_test_values
    x_prediction_07_series=pd.Series(x_prediction_07)
    y_real_07_series=pd.Series(y_real_07)
    ######SHAP#####
    data_SHAP = raw_data
    SHAP_INPUT=data_SHAP.iloc[:,0:14]
    SHAP_OUTPUT=data_SHAP.iloc[:,14]
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
#     explainer = shap.TreeExplainer(best_model,X_SHAP)
    cols=[ 
                         'Zn Content (wt. %)',#0
                      'Co Cotent (wt. %)',#1
                      'Fe Cotent (wt. %)',#2
                      'O Content (at. %)',#4
                      'S Content (at. %)',#5
                      'Content of Pyridinic Species (at. %)',#5
                      'Content of Graphitic Species (at. %)',#6
                      'Content of Oxidized Species (at. %)',#7
                      'Raman ID/IG Ratio',#8
                      'BET Surface Area (m2 g-1)',#9
                      'Micropore Ratio',#10
                      'Mesopore Ratio',#11
                      'Catalyst Loading on RDE/RRDE(mg cm-2)',#12
                      'Rotating Speed (rpm)'
                            ]
    shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), 
                                                                     type="ShapValues")
    shap_values=shap_values[:,:-1]
    shap.summary_plot(shap_values, SHAP_INPUT)

Using matplotlib backend: Qt5Agg


In [4]:
model_lgbmRegressor = lightgbm.LGBMRegressor(random_state=1)
param_lgbm={'boosting_type': ['gbdt'], 'learning_rate': [0.02], 'n_estimators': [100], 'reg_alpha': [0], 'reg_lambda': [0.01], 'subsample': [0.5]}
shap_plot(model_lgbmRegressor,param_lgbm)

start
<class 'numpy.ndarray'>
finished


In [5]:
model_XGBRegressor = xgb.XGBRegressor(random_state=1)
param_xgb={'booster': ['gbtree'], 'learning_rate': [0.1], 'max_depth': [7], 'n_estimators': [800], 'reg_alpha': [0.001], 'reg_lambda': [0.01], 'subsample': [0.8]}
shap_plot(model_XGBRegressor,param_xgb)

start
<class 'numpy.ndarray'>
finished


In [6]:
model_CatboostRegressor=catboost.CatBoostRegressor(random_state=1,verbose=0)
param_cat = {'learning_rate': [0.1], 'max_depth': [8], 'n_estimators': [400], 'subsample': [0.6]} 
shap_plot_CAT(model_CatboostRegressor,param_cat)

start


In [7]:
model_GradientBoostingRegressor = ensemble.GradientBoostingRegressor(random_state=1)
param_GB={'criterion': ['friedman_mse'], 'learning_rate': [0.01], 'loss': ['ls'], 'max_features': ['sqrt'], 'n_estimators': [400]}
shap_plot(model_GradientBoostingRegressor,param_GB)

start
<class 'numpy.ndarray'>
finished


In [8]:
model_RandomForestRegressor = ensemble.RandomForestRegressor(random_state=1)
param_RF = {
    'criterion': ['mse'], 'max_features': ['sqrt'], 'n_estimators': [800]}
shap_plot(model_RandomForestRegressor,param_RF)

start


 94%|=================== | 211/225 [00:15<00:00]       

<class 'numpy.ndarray'>
finished


In [9]:
model_ExtraTreeRegressor = ExtraTreeRegressor(random_state=1)
param_ET = {'criterion': ['friedman_mse'], 'max_depth': [5], 'max_features': ['auto'], 'splitter': ['random']}
shap_plot(model_ExtraTreeRegressor,param_ET)

start
<class 'numpy.ndarray'>
finished


In [10]:
model_DecisionTreeRegressor = tree.DecisionTreeRegressor(random_state=1)
param_DT = {
         'criterion': ['friedman_mse'], 'max_depth': [5], 'max_features': ['auto'], 'splitter': ['random']}
shap_plot(model_DecisionTreeRegressor,param_DT)

start
<class 'numpy.ndarray'>
finished


In [11]:
model_AdaboostRegressor=ensemble.AdaBoostRegressor(random_state=1)
param_Ada = {'learning_rate': [0.05], 'loss': ['exponential'], 'n_estimators': [200]} 
shap_plot(model_AdaboostRegressor,param_Ada)

start
<class 'numpy.ndarray'>
finished
